In [21]:
import pandas as pd
pd.set_option('display.max_rows', None)
import re
from nltk.tokenize import sent_tokenize, word_tokenize

In [22]:
# Read in the train_rev1 datafile downloaded from kaggle
df = pd.read_csv('new_combined_results.csv')
df.head(2)

,id,title,company,description,source,location,min_salary,max_salary
0,1,Senior Software Engineer - Remote,Akamai,**Are you a software developer who loves solvi...,indeed.com,United States,"$113,430","$170,043"
1,2,Software Engineer II,Microsoft,Microsoft’s Azure Data engineering team is lea...,indeed.com,United States,"$94,300","$182,600"


In [23]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yuliiaantonova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
def convert_utf8(s):
    return str(s)

df['description'] = df['description'].map(convert_utf8)

In [33]:
# Define a function to extract experience information
def extract_experience(text):
    # Tokenize the text into sentences and words
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]

    # Define a pattern to match "2 years," "2+ years," "3 years," "5 years," etc.
    experience_pattern = r'(\d+)(\s*[\+]?|[\+]\s*)years'

    # Extract years of experience from the tokenized text
    years_of_experience = None
    for sentence_words in words:
        sentence = ' '.join(sentence_words)
        match = re.search(experience_pattern, sentence, re.IGNORECASE)
        if match:
            years_of_experience = match.group(0)
            break

    return years_of_experience

# Apply the function to the "description" column and create a new column "experience"
df['experience'] = df['description'].apply(extract_experience)

# Display the modified DataFrame
# print(df['experience'])

In [34]:
# Define a function to extract experience information
def extract_experience(text):
    # Tokenize the text into sentences and words
    sentences = sent_tokenize(text)
    words = [word_tokenize(sentence) for sentence in sentences]

    # Define a pattern to match "2 years," "2+ years," "3 years," "5 years," etc.
    experience_pattern = r'(\d+)(\s*[\+]?|[\+]\s*)years'

    # Extract years of experience from the tokenized text
    years_of_experience = None
    for sentence_words in words:
        sentence = ' '.join(sentence_words)
        match = re.search(experience_pattern, sentence, re.IGNORECASE)
        if match:
            years_of_experience = int(match.group(1))
            break

    return years_of_experience

# Apply the function to the "description" column and create a new column "experience"
df['experience'] = df['description'].apply(extract_experience)

# Define a function to categorize experience levels
def categorize_experience(row):
    title = row['title']
    experience = row['experience']

    if not pd.isna(experience):
        if 'software' in title.lower():
            if experience <= 2:
                return "Junior Software Engineer"
            elif 3 <= experience <= 4:
                return "Middle Software Engineer"
            else:
                return "Senior Software Engineer"
        elif 'data' in title.lower():
            if experience <= 2:
                return "Junior Data Engineer"
            elif 3 <= experience <= 4:
                return "Middle Data Engineer"
            else:
                return "Senior Data Engineer"
    return "Other"

# Apply the function to categorize experience levels and create a new column "experience_level"
df['experience_level'] = df.apply(categorize_experience, axis=1)

# Display the modified DataFrame

# print(df['experience_level'])

In [35]:
# a = df.to_dict('records')